Load Cleaned Datasets

In [1]:
import pandas as pd

# Load cleaned datasets
admissions = pd.read_csv(r"D:\Unified Mentor Intership\Data Science projects\Tobacco Use and Mortality, 2004-2015\Cleaned_Datasets\admissions_cleaned.csv")
fatalities = pd.read_csv(r"D:\Unified Mentor Intership\Data Science projects\Tobacco Use and Mortality, 2004-2015\Cleaned_Datasets\fatalities_cleaned.csv")
metrics = pd.read_csv(r"D:\Unified Mentor Intership\Data Science projects\Tobacco Use and Mortality, 2004-2015\Cleaned_Datasets\metrics_cleaned.csv")
prescriptions = pd.read_csv(r"D:\Unified Mentor Intership\Data Science projects\Tobacco Use and Mortality, 2004-2015\Cleaned_Datasets\prescriptions_cleaned.csv")
smokers = pd.read_csv(r"D:\Unified Mentor Intership\Data Science projects\Tobacco Use and Mortality, 2004-2015\Cleaned_Datasets\smokers_cleaned.csv")


Aggregate / Prepare Datasets for Merge
We will aggregate values per year (since only smokers may have age-groups).



Prepare Smoking Rates (average of Male + Female)

In [2]:
# Use 16 and Over group
smoking = smokers.groupby("Year")[["16 and Over"]].mean().reset_index()
smoking.rename(columns={"16 and Over": "smoking_rate"}, inplace=True)


Prepare Admissions & Fatalities

In [3]:
# Total admissions per year
admission_rate = admissions.groupby("Year")["Value"].sum().reset_index()
admission_rate.rename(columns={"Value": "total_admissions"}, inplace=True)

# Total fatalities per year
fatality_rate = fatalities.groupby("Year")["Value"].sum().reset_index()
fatality_rate.rename(columns={"Value": "total_fatalities"}, inplace=True)


Prepare Prescriptions

In [4]:
prescription_df = prescriptions[["Year", "All Pharmacotherapy Prescriptions"]].copy()
prescription_df.rename(columns={"All Pharmacotherapy Prescriptions": "total_prescriptions"}, inplace=True)


Merge All DataFrames

In [5]:
# Merge on Year
merged_df = smoking.merge(admission_rate, on="Year", how="inner")
merged_df = merged_df.merge(fatality_rate, on="Year", how="inner")
merged_df = merged_df.merge(prescription_df, on="Year", how="inner")
merged_df = merged_df.merge(metrics, on="Year", how="left")


Feature Engineering

In [6]:
# Change in smoking rate (year-over-year)
merged_df["smoking_rate_change"] = merged_df["smoking_rate"].diff()

# Prescription rate per 100k smokers (optional)
merged_df["prescription_rate"] = merged_df["total_prescriptions"] / merged_df["smoking_rate"]

# Fatality ratio (deaths per smoker percent)
merged_df["fatality_ratio"] = merged_df["total_fatalities"] / merged_df["smoking_rate"]


Final Check

In [7]:
print(merged_df.head())
merged_df.to_csv("merged_features.csv", index=False)


   Year  smoking_rate  total_admissions  total_fatalities  \
0  2006          22.0        16274948.0         1631446.0   
1  2007          21.0        16736959.0         1616198.0   
2  2008          21.5        21575148.0         1629797.0   
3  2009          21.0        18083462.0         1570432.0   
4  2010          20.5        18563938.0         1560699.0   

   total_prescriptions  Tobacco Price Index  Retail Prices Index  \
0                 2079                713.7                296.4   
1                 2475                751.5                309.1   
2                 2263                784.7                321.3   
3                 2483                815.9                319.7   
4                 2564                878.3                334.5   

   Tobacco Price Index Relative to Retail Price Index  \
0                                              240.8    
1                                              243.1    
2                                              244.2 